# TF-IDF using sklearn

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import time
import string
import pandas as pd
import numpy as np

In [7]:
def text_process(text):
    if type(text)!=str:
        return ''
    res = []
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    return tokens
data = pd.read_csv('fake_job_postings_most_freq_text_cleaned.csv')
data['text'] = data['c_requirements']+' '+data['c_title']+' '+data['c_company_profile']+' '+data['c_benefits']+' '+data['c_description']
data['text_process'] = data['text'].apply(lambda x: " ".join(text_process(x)))

In [8]:
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import TfidfTransformer 
corpus = data['text_process'].tolist()
vectorizer = CountVectorizer()  
X = vectorizer.fit_transform(corpus)  
word = vectorizer.get_feature_names_out()  
transformer = TfidfTransformer()   
tfidf = transformer.fit_transform(X)  

In [9]:
print(len(word))
print(tfidf.shape)

96706
(17880, 96706)


In [10]:
print(word[100:200])

['aandroid' 'aangeboden' 'aangedreven' 'aangepaste' 'aangevuld'
 'aanmelden' 'aanp' 'aanpak' 'aanspreekpunt' 'aansprekende' 'aansturen'
 'aantal' 'aantoonbaar' 'aantoonbare' 'aantrekkelijke' 'aanvang' 'aap'
 'aapc' 'aapcsupervisory' 'aarhus' 'aarki' 'aarna' 'aaron' 'aart' 'aashto'
 'aat' 'aatsome' 'ab' 'aba' 'abaadefbaddbaddfddceeada'
 'abaafadccbecfcdbaebcb' 'ababdaedbdfeebfebbebeeb'
 'ababddcafdeacdacebefdeef' 'abaeacacbebfdfdebcdddcccfeeafa'
 'abaeaeadfbcbbefdbdfea' 'abaebddfdfbcdfcbacafdfecee'
 'abaeceeacefdbebecfedcc' 'abafbeefcebacaafaadceeeecbad'
 'abafcdfddefcebbfeadebcea' 'abakus' 'abandon' 'abandoned' 'abandoning'
 'abandonment' 'abap' 'abapbw' 'abaqus' 'abasic' 'abatement'
 'abbaecadfafeefcfcaebb' 'abbaeccbcdfbecdbdbcdeacf'
 'abbaefefdaeadbaeecbabc' 'abbbcfeedeacebfdfdcaaeddffba'
 'abbbfcdeaeeeeebfbbaacecb' 'abbbfddfbcbbfdbcedbefbdaecfcaa'
 'abbcafaacbdeceeffaaecefbfae' 'abbcdcbabdeefaacfacaadb'
 'abbdaaaabdaefcecfaabfcaaeefec' 'abbdbacbbcbbeaaeaadaedeefebffcb'
 'abbeaafbcfb

In [11]:
print(word[96705])

ﬂexible


In [12]:
print(tfidf[0])

  (0, 93162)	0.10898986394280942
  (0, 92936)	0.05886820979124162
  (0, 92249)	0.034081272052155914
  (0, 92067)	0.025369158536223388
  (0, 91796)	0.07896384597101856
  (0, 91605)	0.0435380411587999
  (0, 91397)	0.02017947821746793
  (0, 91264)	0.042367247237503826
  (0, 91115)	0.057533889540402644
  (0, 90890)	0.02506011430624949
  (0, 90412)	0.045608172391399485
  (0, 89549)	0.02933613270870347
  (0, 89145)	0.024253837028028573
  (0, 88496)	0.06707011824429673
  (0, 87840)	0.07225831361522822
  (0, 87751)	0.040864425059877994
  (0, 86466)	0.028164325186763452
  (0, 86100)	0.03304719835514315
  (0, 85697)	0.0315775479037251
  (0, 85146)	0.08518172565891614
  (0, 84268)	0.01873289830279049
  (0, 83952)	0.062075542227526405
  (0, 83339)	0.026124106104560813
  (0, 83043)	0.06356359129063241
  (0, 82830)	0.028499876999591986
  :	:
  (0, 13572)	0.06548109625758328
  (0, 12632)	0.05955462897291756
  (0, 11209)	0.029598300616047922
  (0, 10930)	0.08043469344314644
  (0, 10675)	0.016668524570

# TF-IDF using Hadoop

In [57]:
text = data.loc[:,['text_process']]
text.to_csv('text.txt',sep = '\t', header = None)

In [58]:
output1_path = "TFIDF_Hadoop/output1/part-r-00000"
pd.read_csv(output1_path,sep='\t',header = None)

,0,1
0,a_10171,1
1,a_10665,1
2,a_10676,1
3,a_10680,1
4,a_10681,1
...,...,...
3113868,ﬁnd_12302,2
3113869,ﬁnd_14841,2
3113870,ﬁnd_233,1
3113871,ﬁnd_2345,1


In [59]:
output2_path = "TFIDF_Hadoop/output2/part-r-00000"
pd.read_csv(output2_path,sep='\t',header = None)

,0,1
0,pr_0,1/256
1,professional_0,1/256
2,offer_0,1/256
3,themcheerful_0,1/256
4,juggler_0,1/256
...,...,...
3113868,independently_9999,2/598
3113869,growth_9999,1/598
3113870,pm_9999,2/598
3113871,apartment_9999,1/598


In [61]:
output3_path = "TFIDF_Hadoop/output3/part-r-00000"
pd.read_csv(output3_path,sep='\t',header = None)

,0,1
0,a@15381,0.006684
1,a@5220,0.009767
2,a@12270,0.006848
3,a@13242,0.011507
4,a@10171,0.007099
...,...,...
3113868,ﬁnd@233,0.008860
3113869,ﬁnd@2345,0.007669
3113870,ﬁnd@14841,0.020800
3113871,ﬁnd@12302,0.020507


# SVM

In [41]:
from sklearn import svm
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(tfidf, data.fraudulent, test_size = 0.2, random_state = 0)

In [29]:
start = process_time()
clf = svm.SVC()
clf.fit(X_train,Y_train)

SVC()

In [31]:
Y_predict = clf.predict(X_test)
accuracy_score(Y_test, Y_predict)

0.9854586129753915

In [33]:
print(classification_report(Y_test,Y_predict,target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      3423
           1       0.97      0.68      0.80       153

    accuracy                           0.99      3576
   macro avg       0.98      0.84      0.90      3576
weighted avg       0.99      0.99      0.98      3576



# PSVM